# Run: pviro5

```
Lead     : ababaian
Issue    : 
Version  : v0.3.5-dev : ab-dev branch
start    : 2020 08 28
complete : 2020 .. ...
files    : ~/serratus/notebook/200828_ab/
s3_files : s3://serratus-public/notebook/200822_ab/
output   : s3://serratus-public/out/200828_pviro5/
```

### Intro/Objectives

- protref5 used as reference


### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/200828_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/200828_ab/'

# date and version
date
git rev-parse HEAD # commit version

Sat Aug 29 00:22:28 PDT 2020
3af0bc63b69402671f33246f2b05a265e448692b


In [2]:
# SRA Accession Initialize
cd $WORK
BATCH='viro_SraRunInfo.csv'

aws s3 cp s3://lovelywater2/sra/viro_SraRunInfo.csv.gz ./
gzip -d *

download: s3://lovelywater2/sra/viro_SraRunInfo.csv.gz to ./viro_SraRunInfo.csv.gz


### Terraform Initialize

In [3]:
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# LINE 153
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..9b9cbd9 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,67 +89,72 @@ re

In [4]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [5]:
cd $WORK
BATCH='viro_SraRunInfo.csv'
wc -l $WORK/$BATCH

22252 /home/artem/serratus/notebook/200828_ab/viro_SraRunInfo.csv


In [7]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200828_ab/viro_SraRunInfo.csv
  date: Sat Aug 29 00:39:16 PDT 2020
  wc  : 22252 /home/artem/serratus/notebook/200828_ab/viro_SraRunInfo.csv
  md5 : e9222b54cee8a65bc3781589f5cbf642  /home/artem/serratus/notebook/200828_ab/viro_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
0b9378e14fd8eea9369e116aaad2ea32  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
c6561dd4be371e42849deb1c4a7fbbe2  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
2252 tmp.chunk02_sraRunInfo.csv
51ca1142ed79386dd6f3afcbf94ce80e  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":2250,"total_rows":22250}


 uploadSRA complete.


## Run Serratus

In [6]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json

cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 25,
  "ALIGN_SCALING_CONSTANT": 0.05,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 1000,
  "CLEAR_INTERVAL": 600,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 10,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 250,
  "GENOME": "protref5",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 5,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 10,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 45
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   549    0     0  100   549      0   2704 --:--:-- --:--:-- --:--:--  2691{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":25,"ALIGN_SCALING_CONSTANT":0.05,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":1000,"CLEAR_INTERVAL":600,"DL_ARGS":"","DL_MAX_I

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Nuke Shutdown
aws ec2 describe-instances \
  --filter Name=tag:Name,Values=serratus-align-instance \
  > align_instances.json

jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
  | pv -l \
  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

In [ ]:
# Exploring sumbler data
# DO NOT UNCOMMENT WILL DELETE MANY FILES
# aws s3 sync s3://serratus-public/out/200823_test/sumbler/ ./
# find . -size -2k -exec rm '{}' \;
# grep 'SRAsum' * | sed 's/;/\t  /g' -

In [8]:
## Done
## 22009 SRA files processed